In [1]:
from pycocotools.cocoeval import COCOeval
import json
import torch
import transform as transf
from torchvision.transforms import Compose
import yaml
from dataset import MedicalBboxDataset
from model import ResNet50
import numpy as np
from utils import bbox_collate, data2target, calc_confusion_matrix, draw_graph

In [85]:
def evaluate_coco_weak(val, threshold=0.05):
    config = yaml.safe_load(open('./config.yaml'))
    dataset_means = json.load(open(config['dataset']['mean_file']))
    dataset_all = MedicalBboxDataset(
        config['dataset']['annotation_file'],
        config['dataset']['image_root'])
    if 'class_integration' in config['dataset']:
        dataset_all = dataset_all.integrate_classes(
            config['dataset']['class_integration']['new'],
            config['dataset']['class_integration']['map'])
    
    transform = Compose([
        transf.ToFixedSize([config['inputsize']] * 2),  # inputsize x inputsizeの画像に変換
        transf.Normalize(dataset_means['mean'], dataset_means['std']),
        transf.HWCToCHW()
        ])

    dataset = dataset_all.split(val, config['dataset']['split_file'])
    dataset.set_transform(transform)
    
    model = ResNet50()
    model.load_state_dict(torch.load(f"/data/unagi0/masaoka/resnet50_classify2.pt"))
    

    results = []
    image_ids = []
    for index in range(len(dataset)):
        data = dataset[index]
        data['img'] = torch.from_numpy(data['img']) #.permute(2, 0, 1)
        # run network
        if torch.cuda.is_available():
            #scores, labels, boxes = model(data['img'].cuda().float().unsqueeze(dim=0))
            scores, labels, boxes = model(data['img'].unsqueeze(0).float(), e=True, aug=False)
        else:
            scores, labels, boxes = model(data['img'].float().unsqueeze(dim=0), e=True, aug=True)
        scores = scores.cpu()
        labels = labels.cpu()
        boxes  = boxes.cpu()

        

        if boxes.shape[0] > 0:
            # change to (x, y, w, h) (MS COCO standard)
            boxes[:, 2] -= boxes[:, 0]
            boxes[:, 3] -= boxes[:, 1]

            # compute predicted labels and scores
            #for box, score, label in zip(boxes[0], scores[0], labels[0]):
            for box_id in range(boxes.shape[0]):
                score = float(scores[box_id])
                label = int(labels[box_id])
                box = boxes[box_id, :]
                

                # scores are sorted, so we can break
                if score < threshold:
                    break

                # append detection for each positively labeled class
                image_result = {
                        'image_id'    : dataset.imgids[index],
                        'category_id' : dataset.label_to_coco_label(label),
                        'score'       : float(score),
                        'bbox'        : box.tolist(),
                    }

                # append detection to results
                results.append(image_result)

        # append image to list of processed images
        image_ids.append(dataset.imgids[index])

        # print progress
        print('{}/{}'.format(index+1, len(dataset)), end='\r')

    if not len(results):
            print("error")
            return
        # write output
    json.dump(results, open(f'/data/unagi0/masaoka/retinanet/opt{val}.json', 'w'), indent=4)

    # load results in COCO evaluation tool
    coco_true = dataset.coco
    coco_pred = coco_true.loadRes(f'/data/unagi0/masaoka/retinanet/opt{val}.json')

    # run COCO evaluation
    coco_eval = COCOeval(coco_true, coco_pred, 'bbox')
    coco_eval.params.imgIds = image_ids
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    return

In [ ]:
for i in range(5):
    evaluate_coco_weak(i)

loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
Loading and preparing results...
DONE (t=0.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.49s).
Accumulating evaluation results...
DONE (t=0.26s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.00

In [2]:
def evaluate_classify(val, threshold=0.05):
    config = yaml.safe_load(open('./config.yaml'))
    dataset_means = json.load(open(config['dataset']['mean_file']))
    dataset_all = MedicalBboxDataset(
        config['dataset']['annotation_file'],
        config['dataset']['image_root'])
    if 'class_integration' in config['dataset']:
        dataset_all = dataset_all.integrate_classes(
            config['dataset']['class_integration']['new'],
            config['dataset']['class_integration']['map'])
    
    transform = Compose([
        transf.ToFixedSize([config['inputsize']] * 2),  # inputsize x inputsizeの画像に変換
        transf.Normalize(dataset_means['mean'], dataset_means['std']),
        transf.HWCToCHW()
        ])

    dataset = dataset_all.split(val, config['dataset']['split_file'])
    dataset.set_transform(transform)
    dataloader_val = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=False, 
                                                num_workers=4, collate_fn=bbox_collate)
    model = ResNet50()
    model.load_state_dict(torch.load(f"/data/unagi0/masaoka/resnet50_classify2.pt"))
    ite = 0
    gt = [0,0,0,0]
    tpa = np.zeros(3)
    fpa = np.zeros(3)
    tna = 0
    fna = np.zeros(3)
    with torch.no_grad():
        for i, d in enumerate(dataloader_val):
            scores = torch.sigmoid(model(d['img'].cuda().float()))
            output = scores.cpu().data.numpy()
            output = np.where(output>0.5,1,0)
            target, n, t, v, u = data2target(d, torch.from_numpy(output))
            target = target.cpu().data.numpy()
            gt = np.array([n,t,v,u])
            tp, fp, fn, tn = calc_confusion_matrix(output, target, gt)
            tpa += tp
            fpa += fp
            fna += fn
            tna += tn
                
            print(f'{i}/{len(dataloader_val)}', end = '\r')
    print(gt,tpa,fpa,tna,fna)
    return gt,tpa,fpa,tna,fna

In [3]:
evaluate_classify(2)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
[69  0  0 69] [797.  73.  68.] [409. 153.  42.] 6932 [23. 21.  8.]


(array([69,  0,  0, 69]),
 array([797.,  73.,  68.]),
 array([409., 153.,  42.]),
 6932,
 array([23., 21.,  8.]))

In [64]:
tpa,fpa,tna,fna  = [[],[],[]], [[],[],[]], [[],[],[]], [[],[],[]],
for i in range(5):
    gt, tp,fp,tn,fn = evaluate_classify(i)
    for i in range(3):
        tpa[i].append(tp[i])
        fpa[i].append(fp[i])
        fna[i].append(fn[i])
        tna[i].append(sum(gt)-tp[i]-fp[i]-fn[i])
print(tpa)
print(fpa)
print(tna)
print(fna)

loading annotations into memory...
Done (t=0.32s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
[982, 75, 76, 5914] [624, 62, 75] [282, 33, 0] [5590] [358, 13, 1]
loading annotations into memory...
Done (t=0.32s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
[914, 141, 76, 6572] [826, 67, 76] [172, 219, 0] [6189] [88, 74, 0]
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
[820, 94, 76, 7379] [792, 71, 76] [380, 6, 0] [6972] [28, 23, 0]
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
creating index...
index created!
creating index...
index created!
[1101, 77, 75, 7781] [1029, 63, 75] [1359, 40, 0] [6342] [72, 14, 0]
loading annotations into memory...
Done (t=0.17s)
creating index...
index created!
creating index...
index created!
creating index...
i

In [1]:
import torch
import numpy as np

In [19]:
a = torch.tensor([0.1,0.5,0.6,0.3]).requires_grad_()
c = torch.tensor([1,1,1,1])
d = torch.tensor([0,0,0,0])
b = torch.where(a>0.5, 1., 0.)
loss = (b).sum()
loss.backward()

TypeError: where(): argument 'input' (position 2) must be Tensor, not float

In [16]:
print(a.grad)

tensor([0.2000, 1.0000, 1.2000, 0.6000])


In [4]:
torch.zeros_like(np.array([1,2]))

TypeError: zeros_like(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [2]:
a = np.array([[1,0,0], 
                        [0,0,0], 
                        [0,1,1]])

b = np.array([[1,0,0],
                        [0,0,0],
                        [0,1,0]])
result = np.all((a-b)==0,axis=1)
np.all((1-a)*(1-b), axis = 1).sum()

1

In [7]:
torch.sigmoid(torch.from_numpy(a).float())

tensor([[0.7311, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000],
        [0.5000, 0.7311, 0.7311]])

In [3]:
c = b*a
c

array([[1, 0, 0],
       [0, 0, 0],
       [0, 1, 0]])

In [59]:
c.sum(axis = 0)

array([2, 1, 1])

In [56]:
c[2] = 1

In [62]:
1/(c.sum(axis = 0) + 1)

array([0.33333333, 0.5       , 0.5       ])

In [63]:
c = c.sum(axis=0)

In [34]:
c=np.array([[1,0,9],
                        [0,0,0],
                        [0,1,10]])

In [36]:
c.sum(axis=1)

array([10,  0, 11])

In [66]:
c

array([2, 1, 1])

In [67]:
c[0]

2

In [83]:
a = torch.tensor([1,2]*5).reshape(-1,2)

In [84]:
a

tensor([[1, 2],
        [1, 2],
        [1, 2],
        [1, 2],
        [1, 2]])

In [87]:
a=np.array([1])
a.ndim

1

In [74]:
import numpy as np

In [78]:
a = np.array([1,3,2])
b = 0
b=1

In [81]:
0/(a+3)

array([0., 0., 0.])